In [8]:
import numpy as np
from PIL import Image
import tensorflow as tf

In [9]:
tf.__version__

'2.16.1'

In [10]:
batch_size = 128
img_height = 48
img_width = 48
AUTOTUNE = tf.data.AUTOTUNE

In [11]:
import pathlib
data_dir_train = pathlib.Path('./images/images/train')
data_dir_val = pathlib.Path('./images/images/validation')

In [12]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_train,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale'
  )

Found 28821 files belonging to 7 classes.


In [13]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_val,
  image_size=(img_height, img_width),
  label_mode='categorical',
  batch_size=batch_size,
  color_mode='grayscale'
  )

Found 7066 files belonging to 7 classes.


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, Rescaling, RandomZoom, RandomContrast

rescaling = Sequential([
    Rescaling(1./255),
])

preprocessing_seq = Sequential([
    rescaling,
    RandomFlip(),
    RandomRotation(0.2),
])

In [16]:
new_train_ds = train_ds.map(
    lambda x, y: (preprocessing_seq(x, training=True), y)
)

In [17]:
new_val_ds = val_ds.map(
    lambda x, y: (rescaling(x), y)
)

In [18]:
new_train_ds.prefetch(buffer_size=AUTOTUNE)
new_val_ds.prefetch(buffer_size=AUTOTUNE)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>

In [19]:
num_classes = 7

In [20]:
from tensorflow.keras.layers import Conv2D, Input, Dropout, Dense, Lambda, MaxPool2D, Flatten, BatchNormalization
from tensorflow.nn import local_response_normalization

alexnet = Sequential([
    Input(shape=(48, 48, 1)),
    Conv2D(filters=96, kernel_size=11, strides=4, padding='same', activation='relu'),
    Lambda(local_response_normalization),
    MaxPool2D(pool_size=3, strides=2, padding='same'),
    BatchNormalization(),

    Conv2D(filters=256, kernel_size=5, strides=1, padding='same', activation='relu'),
    Lambda(local_response_normalization),
    MaxPool2D(pool_size=3, strides=2,  padding='same'),
    BatchNormalization(),

    Conv2D(filters=384, kernel_size=3, strides=1, padding='same', activation='relu'),
    Conv2D(filters=384, kernel_size=3, strides=1, padding='same', activation='relu'),
    Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Flatten(),
    Dense(units=4096, activation='relu'),
    Dense(units=4096, activation='relu'),
    Dropout(0.2),
    Dense(units=num_classes, activation='softmax')
])

In [21]:
alexnet.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 12, 12, 96)     │        11,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 12, 12, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 6, 6, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 6, 6, 96)       │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 6, 6, 256)      │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_1 (Lambda)               │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 384)      │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 3, 3, 384)      │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 3, 3, 256)      │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │     9,441,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │        28,679 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,977,671 (114.36 MB)

 Trainable params: 29,976,455 (114.35 MB)

 Non-trainable params: 1,216 (4.75 KB)

In [22]:
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import AUC, CategoricalAccuracy

alexnet.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', AUC(curve='ROC')])

In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint

epochs = 100

alexnet.fit(
    new_train_ds, 
    validation_data=new_val_ds,
    epochs=epochs,
    callbacks=[ModelCheckpoint('./weights/alexnet.weights.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=True, mode='max')]
)

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 77s 327ms/step - accuracy: 0.2314 - auc: 0.6345 - loss: 2.7247 - val_accuracy: 0.1462 - val_auc: 0.6153 - val_loss: 1.8424
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 72s 320ms/step - accuracy: 0.2488 - auc: 0.6602 - loss: 1.7967 - val_accuracy: 0.2656 - val_auc: 0.6557 - val_loss: 1.8008
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 73s 323ms/step - accuracy: 0.2512 - auc: 0.6716 - loss: 1.7818 - val_accuracy: 0.2819 - val_auc: 0.6958 - val_loss: 1.7511
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 72s 319ms/step - accuracy: 0.2636 - auc: 0.6814 - loss: 1.7671 - val_accuracy: 0.2723 - val_auc: 0.6895 - val_loss: 1.7892
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 72s 319ms/step - accuracy: 0.2855 - auc: 0.6954 - loss: 1.7417 - val_accuracy: 0.3097 - val_auc: 0.7139 - val_loss: 1.7113
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 72s 319ms/step - accuracy: 0.3080 - auc: 0.7074 - loss: 1.7136 - val_accuracy: 0.2969 - val_auc: 0.6956 - val_loss: 1.7307
Epoch 7/100
226/

In [26]:
new_model = Sequential([
    Input(shape=(48, 48, 1)),
    Conv2D(filters=96, kernel_size=11, strides=4, padding='same', activation='relu'),
    Lambda(local_response_normalization),
    MaxPool2D(pool_size=3, strides=2, padding='same'),
    BatchNormalization(),

    Conv2D(filters=256, kernel_size=5, strides=1, padding='same', activation='relu'),
    Lambda(local_response_normalization),
    MaxPool2D(pool_size=3, strides=2,  padding='same'),
    BatchNormalization(),

    Conv2D(filters=384, kernel_size=3, strides=1, padding='same', activation='relu'),
    Conv2D(filters=384, kernel_size=3, strides=1, padding='same', activation='relu'),
    Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Flatten(),
    Dense(units=4096, activation='relu'),
    Dense(units=4096, activation='relu'),
    Dropout(0.2),
    Dense(units=num_classes, activation='softmax')
])

In [27]:
new_model.load_weights('./weights/alexnet.weights.h5')

In [33]:
train_ds.class_names

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [47]:
img = np.asarray(Image.open('./images/validation/angry/842.jpg'))
img = img[np.newaxis, :, :, np.newaxis]
img.shape


(1, 48, 48, 1)

In [48]:
pred = new_model.predict(img)
np.argmax(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


4